In [206]:
from IPython.display import display, HTML
import os
DUMMYSPACE = '_'*80
#import json
#current_location = os.getcwd()
#os.chdir('problems/set1')
#os.listdir()

## Templates
Pakuriam keletą automatinių HTML skriptų generatorių gražiam dizainui

In [207]:
class CustomHTML:
    '''various operations of template scripts kept line by line'''
    # by https://www.w3.org/community/webed/wiki/HTML/Training/Tag_syntax
    # https://www.thoughtco.com/html-singleton-tags-3468620
    directory = 'problems/set1'
    tab = 4
    void = ('area', 'base', 'br', 'col', 'command', 'embed', 'hr', 'img', 'input', 
            'keygen', 'link', 'meta', 'param', 'source', 'track', 'wbr')
       
    def __init__(self):
        pass
    
    def tabbing(self, text):
        return '\n'.join([' '*self.tab + line for line in text.split('\n')])
    
    def element(self, tag_name, content='', **options):
        attr_space = ' '.join(f'{key}="{val}"' for key, val in options.items())
        if attr_space: 
            start_tag = f'<{tag_name} {attr_space}>'
        else: 
            start_tag = f'<{tag_name}>'
            
        if tag_name in self.void:
            return start_tag
        else:
            end_tag = f'</{tag_name}>'
            if type(content) is str:
                return f'{start_tag} {content} {end_tag}'
            else:
                content = "\n".join([self.tabbing(c) for c in content])
                return f'{start_tag}\n{content}\n{end_tag}'
    
    def hjoin(self, items):
        text = '\n&emsp;\n'.join([self.element('div', [item]) for item in items])
        return self.element('div', [text], style='display: flex')
        
    def vjoin(self, items, sep='\n&emsp;\n'):
        return sep.join(items)
    
    def itemize(self, content):
        if type(content) is str:
            raise TypeError('You should itemize bullet list, not str')
        return self.element('ul', [self.element('li', c) for c in content])
    
    def duota_rasti(self, duota, rasti):
        return self.vjoin([self.element('h4', 'DUOTA:'), self.itemize(duota),
                          self.element('h4', 'RASTI:'), self.itemize(rasti)], sep='\n')
    
    def brezinys_duota_rasti(self, brezinys, duota, rasti, setup=dict()):
        return self.hjoin([self.element('img', 
                                       src=os.path.join(self.directory, brezinys), 
                                       style="height: 200px"),
                         self.duota_rasti(duota, rasti)])
        
    def sprendimas(self, steps):
        return self.vjoin([self.element('h3', 'SPRENDIMAS:'), self.itemize(steps)], sep='\n')
    
print(CustomHTML().brezinys_duota_rasti('my_draw.jpg', ['a', 'b'], ['c']))

<div style="display: flex">
    <div>
        <img src="problems/set1/my_draw.jpg" style="height: 200px">
    </div>
    &emsp;
    <div>
        <h4> DUOTA: </h4>
        <ul>
            <li> a </li>
            <li> b </li>
        </ul>
        <h4> RASTI: </h4>
        <ul>
            <li> c </li>
        </ul>
    </div>
</div>


Pora pavyzdukų, kaip jie veikia:`

In [201]:
CustomHTML.directory='problems/set1'
script = CustomHTML().brezinys_duota_rasti('my_draw.jpg', ['a', 'b'], ['c'])
display(HTML(script))

## Uždavinio sandara
* Uždavinys - tai žinomųjų rinkinys $a_1$, $a_2$, ..., $a_n$ kartu su nurodymu pateikti sprendimą, kaip rasti vieną ar kelis nežinomuosius $x_1$, $x_2$, ..., $x_n$. Senesnis vaizdavimas uždavinio kaip Toulmino modeliu pagrįstas žemėlapis buvo ne toks lankstus, nes dažnai taisyklės taikomos keliems duomenims ar jų išvadoms, o jas sukomplektuoti dažnai yra sunku. 
  
* Uždaviniui galima:
  * rasti tarpines sprendimo būsenas turint sprendimo žingsnius (klausimas, sprendimas)
  * paskelbti tam tikrą sprendimo žingsnių rinkinį nauju uždaviniu
  * išbraižyti grafą  
  
Apsirašykim reikiamas klases uždavinio sprendimui

In [208]:
class Data:
    namespace = set()
    targetspace = set()
    stepspace = dict()
    references = dict()
    
    def __init__(self, n, fr='', to='', by=''):
        self.n = n
        self.fr = fr
        self.to = to
        self.by = by
    

#----------------------------------------------------------------------------------------------
# Which kinds of data do I expect during solution process?

class Known(Data):
    '''Descriptor for a known data of text problem refered in condition.
    Responsible for a process of introducing notation if it is required to do so.

    Parameters
    -----
    n: a number of data descriptor instance unique to every instance of Data
    fr: a part of condition that describes a single claim
    to: notation introduced (optional)
    by: general environment required to introduce notation; 
        usually a relation that refers to some objects or quantities used
    '''
    
    def __init__(self, *args, **kwargs):
        Data.__init__(self, *args, **kwargs)
        Data.namespace.add(self.n)
        Data.references[self.n] = self
    
class Unknown(Data):
    '''Descriptor for a data to look for refered in condition.
    Responsible for a process of introducing notation if it is required to do so.

    Parameters
    -----
    n: a number of data descriptor instance unique to every instance of Data
    fr: a part of condition that refers what to look for
    to: notation introduced (optional)
    by: general environment required to introduce notation; 
        usually an object or quantity
    '''
    
    def __init__(self, *args, **kwargs):
        Data.__init__(self, *args, **kwargs)
        Data.targetspace.add(self.n)
        Data.references[self.n] = self
        
class IfThen(Data):
    '''Descriptor for a derived data from data found or given already
    Responsible for a process of deriving a new data using rules and definitions

    Parameters
    -----
    n: a number of data descriptor instance unique to every instance of Data
    fr: a minimal set of data needed to derive a current claim
    to: notation introduced (optional)
    by: general environment that is a rationale for deriving a current claim; 
        usually a main rule or definition to apply on a minimal set of data
    ask: a question that includes a minimal set of data and asks to derive a current claim
    background: additional environment that is to weak intuitively to be interpreted
        as a rationale for deriving a new claim but plays an important role in a
        process in derivation
    '''
    
    def __init__(self, *args, ask='', background=(), **kwargs):
        self.ask = ask
        self.background = background
        Data.__init__(self, *args, **kwargs)
        Data.stepspace[self.n] = self.fr
        Data.references[self.n] = self

#----------------------------------------------------------------------------------------------
# Which kinds of reasoning do I expect during solution process?

class Warrant:
    def __init__(self, *args, **kwargs):
        self.args=args
        self.kwargs=kwargs

class Eq(Warrant):
    def __init__(self, *args, **kwargs):
        Warrant.__init__(self, *args, **kwargs)
        
class Def(Warrant):
    def __init__(self, *args, **kwargs):
        Warrant.__init__(self, *args, **kwargs)
        
class Rule(Warrant):
    def __init__(self, *args, **kwargs):
        Warrant.__init__(self, *args, **kwargs)
      
    
#----------------------------------------------------------------------------------------------
# Which processes and parameters are involved in solution of problem?

class Problem:
    Data.namespace, Data.stepspace = set(), dict()    
    Data.targetspace, Data.references = set(), dict()
    
    def __init__(self, task, data, steps, analogies=None, conclusion=None, author=None, picture=None):
        self.task=task
        self.data=data
        self.steps=steps
        self.analogies=analogies
        self.conclusion=conclusion
        self.author=author
        self.picture=picture
        
        self.metadata={'namespace': Data.namespace,
                       'stepspace': Data.stepspace,
                       'targetspace': Data.targetspace,
                       'references': Data.references}

Pakurkim klasę uždavinio sprendimo procesui aprašyti

In [209]:
pr = Problem(
task = 'Sujungus keturkampio kraštinių vidurio taškus gautas lygiagretainis. Raskite jo perimetrą, jei viena jo įstrižainė lygi $x$, o kita $y$',
data=(
    Known(n=0,
          fr='$X$ - atkarpos $AB$ vidurio taškas', 
          to='$AX=XB$', 
          by=Eq('vidurio taškas')),
    Known(n=1,
          fr='$Y$ - atkarpos $BC$ vidurio taškas', 
          to='$BY=YC$', 
          by=Eq('vidurio taškas')),
    Known(n=2,
          fr='$Z$ - atkarpos $CD$ vidurio taškas', 
          to='$CZ=ZD$', 
          by=Eq('vidurio taškas')),
    Known(n=3,
          fr='$T$ - atkarpos $DA$ vidurio taškas', 
          to='$DT=TA$', 
          by=Eq('vidurio taškas')),
    Known(n=4,
          fr='Viena lygiagretainio įstrižainė lygi $x$',
          to='$AC=x$', 
          by=Eq('atkarpos ilgis', '$x$')),
    Known(n=5,
          fr='Kita lygiagretainio įstrižainė lygi $y$',  
          to='$BD=y$' + DUMMYSPACE, 
          by=Eq('atkarpos ilgis', '$y$')),
    Unknown(n=6,
            fr='Raskite lygiagretainio perimetrą', 
            to='$P_{XYZT}$', 
            by='perimetras')),  
steps=(
    IfThen(n=7, fr=(0, 1),
           to=r'$XY$ yra $\triangle ABC$ vidurio linija', 
           by=Def('vidurio linijos apibrėžimas'),
           ask='Kokia savybe pasižymi atkarpa $XY$?'),
       
    IfThen(n=8, fr=(2, 3),
           to=r'$ZT$ yra $\triangle ACD$ vidurio linija', 
           by=Def('vidurio linijos apibrėžimas'),
           ask='Kokia savybe pasižymi atkarpa $ZT$?'),
    
    IfThen(n=9, fr=(1, 2),
           to=r'$YZ$ yra $\triangle ABD$ vidurio linija', 
           by=Def('vidurio linijos apibrėžimas'),
           ask='Kokia savybe pasižymi atkarpa $YZ$?'),
    
    IfThen(n=10, fr=(0, 3),
           to=r'$XT$ yra $\triangle BCD$ vidurio linija', 
           by=Def('vidurio linijos apibrėžimas'),
           ask='Kokia savybe pasižymi atkarpa $XT$?'),
    
    IfThen(n=11, fr=(7,4),
           to=r'$\displaystyle XY = \frac{x}{2}$', 
           by=Def('vidurio linijos savybė'), 
           ask='$XY$',
           background=(4,)),
    
    IfThen(n=12, fr=(8,4),
           to=r'$\displaystyle ZT = \frac{x}{2}$', 
           by=Rule('vidurio linijos savybė'), 
           ask='$ZT$',
           background=(4,)),
    
    IfThen(n=13, fr=(9,5),
           to=r'$\displaystyle YZ = \frac{y}{2}$', 
           by=Rule('vidurio linijos savybė'), 
           ask='$YZ$',
           background=(5,)),
    
    IfThen(n=14, fr=(10,5),
           to=r'$\displaystyle XT = \frac{y}{2}$', 
           by=Rule('vidurio linijos savybė'), 
           ask='$XT$',
           background=(5,)),
    
    IfThen(n=6, fr=(11,12,13,14), 
           to=r'$P_{XYZT}=XY+ZT+XY+ZT=\frac{x}{2}+\frac{x}{2}+\frac{y}{2}+\frac{y}{2}=x+y$', 
           by=Def('perimetro apibrėžimas'), 
           ask='$P_{XYZT}$',
           background=('trupmenų su vienodais vardikliais sudėtis',11,12,13,14))),
    
analogies = [(0,1,2,3), (4,5), (7,8,9,10), (11,12,13,14)],
conclusion = 'Lygiagretainio, gauto sujungus keturkampio kraštinių vidurio taškus, perimetras lygus to keturkampio įstrižainių sumai',
author = 'vadovėlis gimnazijų 1 ir 2 klasėms',
picture = 'my_draw.jpg')

Svarbiausia informacija automatiškai susidėlioja į klasės `Data` kintamuosius vos tik yra sukuriami `IfThen`, `Known`, `Unknown` kintamieji. Kiekvienąsyk kuriant naują `Problem` egzempliorių ši informacija nusiresetina, o paskui pasipildo reikiamais duomenimis automatiškai.

Alternatively:

In [15]:
#Data.namespace, Data.stepspace, Data.targetspace, Data.references
pr.metadata

{'namespace': {0, 1, 2, 3, 4, 5},
 'stepspace': {7: (0, 1),
  8: (2, 3),
  9: (1, 2),
  10: (0, 3),
  11: (7, 4),
  12: (8, 4),
  13: (9, 5),
  14: (10, 5),
  6: (11, 12, 13, 14)},
 'targetspace': {6},
 'references': {0: <__main__.Known at 0x7f2d1462baf0>,
  1: <__main__.Known at 0x7f2d1462bbe0>,
  2: <__main__.Known at 0x7f2d1462bc70>,
  3: <__main__.Known at 0x7f2d146964f0>,
  4: <__main__.Known at 0x7f2d14696100>,
  5: <__main__.Known at 0x7f2d145700d0>,
  6: <__main__.IfThen at 0x7f2d1457f7f0>,
  7: <__main__.IfThen at 0x7f2d1457fd60>,
  8: <__main__.IfThen at 0x7f2d1457fcd0>,
  9: <__main__.IfThen at 0x7f2d1457fc40>,
  10: <__main__.IfThen at 0x7f2d1457fbb0>,
  11: <__main__.IfThen at 0x7f2d1457faf0>,
  12: <__main__.IfThen at 0x7f2d1457fa30>,
  13: <__main__.IfThen at 0x7f2d1457f970>,
  14: <__main__.IfThen at 0x7f2d1457f8b0>}}

## Realizacija
Kiekvienam sprendimo žigsniui pasižiūri, iš kokių duomenų id jis gautas, o tada nukreipia į jų tekstą (jau sutvarkytą maksimaliai, nepriklausomai nuo to, ar buvo atliktas žymėjimas, ar išvedimas)

In [228]:
def return_subtasks(pr, return_solution=False):
    views = []
    for id, ifthen in enumerate(pr.steps):
        brezinys = 'my_draw.jpg'
        duota = tuple(pr.metadata['references'][ref].to for ref in ifthen.fr)
        rasti = [ifthen.ask]
        
        header = CustomHTML().element('h3', f'SĄLYGA: žingsnis {id+1}')
        sprendimas = CustomHTML().sprendimas([ifthen.to])
        view = CustomHTML().brezinys_duota_rasti(brezinys, duota, rasti)
        if return_solution:
            v = CustomHTML().vjoin([header, view, sprendimas])
        else:
            v = CustomHTML().vjoin([header, view])
        views.append(v)
    return views
        
def return_task(pr, return_solution=False, no_brezinys=False):
    if return_solution:
        steps = [pr.metadata['references'][i].to for i in pr.metadata['stepspace']]
        sprendimas = CustomHTML().sprendimas(steps)
        
    if no_brezinys:
        header = CustomHTML().element('h3', f'SĄLYGA be brėžinio')
        view = pr.task
    else:
        header = CustomHTML().element('h3', f'SĄLYGA su brėžiniu')     
        brezinys = pr.picture
        duota = [pr.metadata['references'][i].to for i in pr.metadata['namespace']]
        rasti = [pr.metadata['references'][i].to for i in pr.metadata['targetspace']]
        view = CustomHTML().brezinys_duota_rasti(brezinys, duota, rasti)
        
    if return_solution:
        return CustomHTML().vjoin([header, view, sprendimas], sep='\n&emsp;\n\n')
    else:
        return CustomHTML().vjoin([header, view], sep='\n&emsp;\n\n')

In [225]:
subtasks = return_subtasks(pr, return_solution=True)
HTML(subtasks[1])

In [233]:
h = return_task(pr, return_solution=True, no_brezinys=False)
HTML(h)